In [ ]:
# >>> Path configuration (auto-inserted) >>>
from pathlib import Path
import os

PROJECT_ROOT = Path(os.getenv("GBB_PROJECT_ROOT", ".")).resolve()
DATA = PROJECT_ROOT / "data"
PATSTAT = PROJECT_ROOT / "patstat"
PATTEXT = PROJECT_ROOT / "patent_text"
SAMPLEDATA = PROJECT_ROOT / "sampledata"

# Fallback to sampledata if primary paths not present
if not PATSTAT.exists() and (SAMPLEDATA / "patstat").exists():
    PATSTAT = SAMPLEDATA / "patstat"
if not PATTEXT.exists() and (SAMPLEDATA / "patent_text").exists():
    PATTEXT = SAMPLEDATA / "patent_text"
if not DATA.exists() and (PROJECT_ROOT / "data").exists():
    DATA = PROJECT_ROOT / "data"

def P(*parts):
    return str(Path(*parts))
# <<< Path configuration (auto-inserted) <<<


In [1]:
import pandas as pd
import numpy as np

In [2]:
tls207 = pd.read_parquet(
    str(Path.home() / "lab/data/patstat/patstat2019/tls207.parquet"),
    columns=["person_id", "appln_id"],
)
tls207.head()

,person_id,appln_id
0,1,1
1,1,7
2,1,46
3,1,775
4,1,1192


In [3]:
tls201 = pd.read_parquet(
    str(Path.home() / "lab/data/patstat/patstat2019/tls201.parquet"),
    columns=["appln_id","docdb_family_id"]
)
tls201.head()

,appln_id,docdb_family_id
0,0,0
1,1,8554171
2,2,27517085
3,3,7915918
4,4,22889365


In [4]:
tls206 = pd.read_parquet(
    str(Path.home() / "lab/data/patstat/patstat2019/tls206.parquet"),
    columns=["person_id","person_ctry_code", "psn_id", "psn_name", "psn_sector"],
)
tls206.head()

,person_id,person_ctry_code,psn_id,psn_name,psn_sector
0,50015510,US,24479907,"PRINCE, MELVIN JAY",<NA>
1,50015511,US,24480361,PRINCETON GAMMA TECH INSTRUMENTS,COMPANY
2,50015512,US,24481402,"PRINGLE, DAVID",<NA>
3,50015513,US,24487507,"PRITCHARD, BYRON ANDREW",<NA>
4,50015514,US,24494825,PROCESS QUERY SYSTEMS,COMPANY


In [5]:
firmidname = tls206.query("psn_sector!='INDIVIDUAL' and psn_sector!='UNKNOWN'").dropna()
inventor = tls206.query("psn_sector=='INDIVIDUAL'").dropna()
inventor.head()

,person_id,person_ctry_code,psn_id,psn_name,psn_sector
5,50015515,US,24497893,"PROCTOR, JOSH",INDIVIDUAL
8,50015518,US,24503265,"PROEST, KARL-HEINZ",INDIVIDUAL
15,50015525,US,24579694,"PUJOS, MURIEL",INDIVIDUAL
20,50015530,US,24625120,"PYTLIK, LORI K. THALMANN",INDIVIDUAL
24,50015534,US,24746515,"QUARANTA, JR., JAMES R.",INDIVIDUAL


In [6]:
firmctry = (firmidname[['person_id','psn_id']]
    .merge(tls207)
    .merge(tls207.rename(columns={'person_id':'invid'}))
    .merge(inventor[['person_id','person_ctry_code']].rename(columns={'person_id':'invid','person_ctry_code':'ctry_code'}))
    .groupby(['psn_id','ctry_code'])['invid'].nunique().reset_index()
)
firmctry.head()

psn_id  ctry_code
22                   1
26      PL           1
33      RU           1
36      ES           1
38      UA           1
Name: invid, dtype: int64

In [7]:
firmctry2 = firmidname.groupby(["psn_id", "psn_name","psn_sector"])['person_ctry_code'].agg(lambda x: pd.Series.mode(x)[0]).reset_index()
firmctry2.head()

,psn_id,psn_name,psn_sector,person_ctry_code
0,2,',COMPANY,RU
1,17,' * CONPROJECT' HANDELSVERTRETUNG UND TECHN BU...,COMPANY,AT
2,19,' 1C' [RU/RU],COMPANY,
3,20,- A. VIGORELLI,COMPANY,IT
4,21,"'' AEROSOLS ANTWERPIA'', ZOERSELBAAN, 25, WEST...",COMPANY,


In [16]:
firmctry = firmctry.drop(columns=['index'])
firmctry.head()

,psn_id,ctry_code,invid
0,22,,1
1,26,PL,1
2,33,RU,1
3,36,ES,1
4,38,UA,1


In [21]:
firmctryagg = firmctry[firmctry.ctry_code!='  '].sort_values('invid',ascending=False).groupby('psn_id').head(1)
firmctryagg.head()

,psn_id,ctry_code,invid
1588128,26564373,KR,75826
892197,12911408,US,38559
1151276,18110090,KR,31265
2060214,31014482,JP,29460
1434754,23284634,JP,28574


In [22]:
firmctryagg = firmctry2.merge(firmctryagg[['psn_id','ctry_code']],how='left')
firmctryagg['ctry_code'] = np.where(firmctryagg['ctry_code'].isna(),firmctryagg['person_ctry_code'],firmctryagg['ctry_code'])
firmctryagg.head()

,psn_id,psn_name,psn_sector,person_ctry_code,ctry_code
0,2,',COMPANY,RU,RU
1,17,' * CONPROJECT' HANDELSVERTRETUNG UND TECHN BU...,COMPANY,AT,AT
2,19,' 1C' [RU/RU],COMPANY,,
3,20,- A. VIGORELLI,COMPANY,IT,IT
4,21,"'' AEROSOLS ANTWERPIA'', ZOERSELBAAN, 25, WEST...",COMPANY,,


In [24]:
firmctryagg.drop(columns=['person_ctry_code']).to_parquet('firmctry.parquet',index=False)

In [25]:
firmpat = (
    tls201.merge(tls207)
    .merge(firmidname[["person_id", "psn_id"]].drop_duplicates())[
        ["psn_id", "docdb_family_id"]
    ]
    .drop_duplicates()
)
firmpat.head()

,psn_id,docdb_family_id
0,22321642,8554171
1,21290306,27517085
2,20233499,27517085
3,24000817,7915918
4,24000671,7915918


In [26]:
tls225 = pd.read_parquet(str(Path.home() / "lab/data/patstat/patstat2019/tls225.parquet"),columns=['docdb_family_id','cpc_class_symbol']).rename(columns={'cpc_class_symbol':'cpc'})
tls225['cpc'] = tls225['cpc'].str.replace(' ','')
tls225.head()

,docdb_family_id,cpc
0,76,E04C1/42
1,184,B23Q16/005
2,277,F16C33/4664
3,277,F16C33/52
4,292,G01F1/07


In [27]:
firmpatcpc = firmpat.merge(
    firmpat.groupby("psn_id")["docdb_family_id"]
    .nunique()
    .reset_index()
    .rename(columns={"docdb_family_id": "totalpat"}).query('totalpat>=100')
).merge(tls225)
firmpatcpc.head()

,psn_id,docdb_family_id,totalpat,cpc
0,22321642,8554171,19095,G06K7/0013
1,22321642,8554171,19095,G06K7/0021
2,22321642,8554171,19095,G06K7/0043
3,22321642,8554171,19095,G06K7/0069
4,22321642,8554171,19095,G06K19/07739


In [28]:
firmpatcpc[['psn_id','docdb_family_id']].nunique()

psn_id                37371
docdb_family_id    13812533
dtype: int64

In [29]:
firmpatcpc.query('totalpat>=200')[['psn_id','docdb_family_id']].nunique()

psn_id                17853
docdb_family_id    12708504
dtype: int64

In [30]:
firmpatcpc.query('totalpat>=500')[['psn_id','docdb_family_id']].nunique()

psn_id                 7217
docdb_family_id    11338759
dtype: int64

In [31]:
firmpatcpc.query('totalpat>=1000')[['psn_id','docdb_family_id']].nunique()

psn_id                 3606
docdb_family_id    10235331
dtype: int64

In [32]:
firmpatcpc.to_parquet('firmpatcpc.parquet',index=False)

In [1]:
import pandas as pd

In [2]:
firmctry = pd.read_parquet('firmctry.parquet')
firmctry.head()

,psn_id,psn_name,psn_sector,ctry_code
0,2,',COMPANY,RU
1,17,' * CONPROJECT' HANDELSVERTRETUNG UND TECHN BU...,COMPANY,AT
2,19,' 1C' [RU/RU],COMPANY,
3,20,- A. VIGORELLI,COMPANY,IT
4,21,"'' AEROSOLS ANTWERPIA'', ZOERSELBAAN, 25, WEST...",COMPANY,


In [3]:
firmctry[firmctry.duplicated('psn_id',keep=False)].sort_values('psn_id')

,psn_id,psn_name,psn_sector,ctry_code
50,75,' NAUCHNO-ISSLEDOVATEL'SKIJ INSTITUT GIDROSVJA...,COMPANY,RU
51,75,' NAUCHNO-ISSLEDOVATEL'SKIJ INSTITUT GIDROSVJA...,GOV NON-PROFIT,RU
174,357,(AGIL)ASIAN GEMMOLOGICAL INSTITUTE AND LABORATORY,COMPANY,
175,357,(AGIL)ASIAN GEMMOLOGICAL INSTITUTE AND LABORATORY,GOV NON-PROFIT,
197,455,(BEIJING)CHINA RAILWAY CONSTRUCTION ELECTRIFIC...,GOV NON-PROFIT,
...,...,...,...,...
4119971,35198258,ZYKOV,COMPANY,RU
4120299,35200931,ZYWIECKA FABRYKA SPRZETU SZPITALNEGO FAMED,COMPANY HOSPITAL,PL
4120298,35200931,ZYWIECKA FABRYKA SPRZETU SZPITALNEGO FAMED,COMPANY,PL
4120301,35200932,ŻYWIECKA FABRYKA SPRZĘTU SZPITALNEGO FAMED SPO...,HOSPITAL,PL


In [4]:
firmctry.drop_duplicates(subset='psn_id').to_parquet('firmctry.parquet',index=False)

In [2]:
df_pat = pd.read_parquet('firmpatcpc.parquet')
df_pat = df_pat[(df_pat.totalpat>=500)&(df_pat.cpc.str[:2]=='Y0')]
df_pat.head()

,psn_id,docdb_family_id,totalpat,cpc
185,23941908,8846615,14049,Y02T10/126
202,1532093,21883779,1001,Y02D70/122
481,747214,35447751,3541,Y02E20/16
796,28259468,39832294,154840,Y02T50/67
797,28259468,39832294,154840,Y02T50/6765


In [3]:
df_pat = df_pat[['psn_id','docdb_family_id']].drop_duplicates()
df_pat['np'] = df_pat.groupby('docdb_family_id')['psn_id'].transform('nunique')
df_pat = df_pat[df_pat['np']>1]
df_pat.head()

,psn_id,docdb_family_id,np
861,26849252,39829036,2
1721,1983640,7672727,2
4573,24000671,32682060,2
5822,25527437,34837535,2
5836,31014482,34960613,2


In [5]:
df_copat = df_pat[['psn_id','docdb_family_id']].rename(columns={'psn_id':'supply'}).merge(df_pat[['psn_id','docdb_family_id']].rename(columns={'psn_id':'demand'})).groupby(['supply','demand'])['docdb_family_id'].nunique().reset_index()
df_copat.head()

,supply,demand,docdb_family_id
0,42082,42082,1
1,42082,11898188,1
2,42082,12487574,1
3,43121,43121,24
4,43121,43160,21


In [6]:
df_copat.supply.nunique()

4469

In [7]:
df_copat.to_parquet('firm_cogreenpat.parquet')